In [1]:
import numpy as np
from scipy.optimize import least_squares

# ---------------------------------------------
# Characteristic function of the Bates model
# ---------------------------------------------
def bates_characteristic_function(u, T, S0, r, q, kappa, theta, sigma_v, rho, v0, 
                                  lambda_j, mu_j, sigma_j):
    """
    Bates model characteristic function for log(S_T).

    Inputs:
        u: complex argument
        T: time to maturity
        S0: initial underlying price
        r: risk-free rate
        q: continuous dividend yield
        model parameters: kappa, theta, sigma_v, rho, v0, lambda_j, mu_j, sigma_j
        
    Returns:
        phi(u): characteristic function value at u.
    """

    # Decompose and define intermediates as per Bates model formula
    
    # The drift correction term for jumps under risk-neutral measure:
    omega_j = np.exp(mu_j + 0.5 * sigma_j**2) - 1.0
    
    # Adjusted jump intensity for risk-neutral measure could be lambda_j,
    # or if under Q measure it might differ. For standard Bates Q parameters,
    # we assume lambda_j is already Q-measure intensity.
    
    # Heston part characteristic exponent:
    # As per standard Heston, define d and g:
    iu = 1j * u
    alpha = -0.5 * (u*u + 1j*u)
    beta = kappa - rho * sigma_v * iu
    gamma = 0.5 * sigma_v**2
    
    d = np.sqrt(beta*beta - 4.0 * alpha * gamma)
    g = (beta - d) / (beta + d)
    
    # Heston characteristic function for variance:
    C = (1.0/gamma) * ( (beta - d) * (T) - 2.0 * np.log((1 - g * np.exp(-d*T)) / (1 - g)) )
    D = (beta - d)/gamma * ( (1 - np.exp(-d*T)) / (1 - g*np.exp(-d*T)) )
    
    # Combine jump part:
    # Jump characteristic exponent:
    # Cf. Jump term: E[e^{iu * (sum of jumps)}] = exp(lambda * T * ((e^{iu * Y} - 1)))
    # For Normal jumps: E[e^{iuY}] = exp(iu mu_j - 0.5 u^2 sigma_j^2)
    # So jump part = exp(lambda_j * T * (exp(iu mu_j - 0.5 sigma_j^2 u^2) - 1))
    jump_char = np.exp(lambda_j * T * (np.exp(iu * mu_j - 0.5 * sigma_j**2 * u**2) - 1.0))
    
    # Full characteristic function:
    phi = np.exp(iu*(np.log(S0) + (r - q)*T) + C*theta - D*v0) * jump_char
    
    return phi

# ---------------------------------------------
# European Call Pricing Using a Fourier Approach (Lewis formula)
# ---------------------------------------------
def bates_call_price(S0, K, T, r, q, params):
    """
    Price a European call using the Bates model characteristic function and
    a Fourier integral approach (Lewis 2001 formula).
    """
    # Unpack parameters:
    kappa, theta, sigma_v, rho, v0, lambda_j, mu_j, sigma_j = params
    
    # For numerical integration, define integrand:
    # Lewis formula for call price:
    # C = S0 * exp(-qT) - K * exp(-rT)/π ∫_0^∞ Re[ (exp(-iu ln K) * phi(u - i)) / (i u * phi(-i)) ] du
    # Here we adapt a commonly used integral representation. 
    # We'll implement the characteristic function shifted by -i for the integrand.
    
    # Characteristic function under transform u - i
    def phi_transform(u):
        return bates_characteristic_function(u - 1j, T, S0, r, q, kappa, theta, sigma_v, rho, v0, lambda_j, mu_j, sigma_j)

    # The integral can be approximated using numerical integration (e.g., Simpson's rule or quad).
    from scipy.integrate import quad

    lnK = np.log(K)

    def integrand(u):
        u_complex = u + 0j
        numerator = np.exp(-1j*u_complex*lnK)*phi_transform(u_complex)
        denominator = 1j*u_complex * bates_characteristic_function(-1j, T, S0, r, q, kappa, theta, sigma_v, rho, v0, lambda_j, mu_j, sigma_j)
        val = numerator/denominator
        return val.real

    integral_result, _ = quad(integrand, 0, 200)  # upper limit might need refinement

    # Final price:
    call_price = S0*np.exp(-q*T) - K*np.exp(-r*T)*integral_result/np.pi
    return call_price

# ---------------------------------------------
# Objective Function for Calibration
# ---------------------------------------------
def calibration_error(params, market_data, S0, r, q):
    # params: [kappa, theta, sigma_v, rho, v0, lambda_j, mu_j, sigma_j]
    errors = []
    for (K, T, market_price) in market_data:
        model_price = bates_call_price(S0, K, T, r, q, params)
        errors.append(model_price - market_price)
    return errors

# ---------------------------------------------
# Example Calibration Routine
# ---------------------------------------------
if __name__ == "__main__":
    # Example (dummy) market data: (K, T, market_call_price)
    market_data = [
        (100, 0.5, 5.0),
        (110, 0.5, 2.5),
        (90,  0.5, 10.0),
        (100, 1.0, 7.0),
        # ... add more data
    ]
    S0 = 100
    r = 0.01
    q = 0.00

    # Initial guess for parameters
    # [kappa, theta, sigma_v, rho, v0, lambda_j, mu_j, sigma_j]
    initial_guess = [2.0, 0.04, 0.5, -0.5, 0.04, 0.1, -0.1, 0.2]

    # Perform least squares calibration
    result = least_squares(calibration_error, initial_guess, 
                           args=(market_data, S0, r, q),
                           bounds=( [1e-5, 1e-5, 1e-5, -0.9999, 1e-5, 1e-5, -10, 1e-5],
                                    [10, 1, 5, 0.9999, 1, 10, 10, 5] ))

    calibrated_params = result.x
    print("Calibrated parameters:")
    print(f"kappa={calibrated_params[0]}, theta={calibrated_params[1]}, sigma_v={calibrated_params[2]}, rho={calibrated_params[3]},")
    print(f"v0={calibrated_params[4]}, lambda_j={calibrated_params[5]}, mu_j={calibrated_params[6]}, sigma_j={calibrated_params[7]}")


/var/folders/5g/b4bpmvtn7tjcmrg4l2k_p_9r0000gn/T/ipykernel_51813/2717787580.py:92: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  integral_result, _ = quad(integrand, 0, 200)  # upper limit might need refinement
/var/folders/5g/b4bpmvtn7tjcmrg4l2k_p_9r0000gn/T/ipykernel_51813/2717787580.py:92: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  integral_result, _ = quad(integrand, 0, 200)  # upper limit might need refinement


Calibrated parameters:
kappa=2.030208519925757, theta=0.04780202723196492, sigma_v=0.5025067131223329, rho=-0.5928308871687656,
v0=0.03972341866806533, lambda_j=1.1443066097733838, mu_j=3.033474671340872, sigma_j=0.005123736620439857


In [3]:
calibrated_params

array([ 2.03020852,  0.04780203,  0.50250671, -0.59283089,  0.03972342,
        1.14430661,  3.03347467,  0.00512374])

In [7]:
param = 5, 0.2**2, 0.001, 0, 0.2**2, 0.1, 0, 0.001 

In [9]:
bates_call_price(100, 100, 1, 0, 0, param)

3.5064202333200734e+25